# 🍽️🍳 مصنف وصفات الطعام 
### 🍔 🍕 🍗 🍖 🥩 🍤 🍛 🍣 🌮 🥙 🍞 🧀 🧁 🍩 🍪 🍫 🍯 🍎 🍉 🍇 🍓 🥝 🍍 🥥 🥑 🥕 🌽 🥔 🥬 🍵 ☕ 🍜 🍲 🍚 🍙 🍘 🥟 🥠 🥡 🥣 🥞 🧇 🍮 🍨 🥞 🧇 🍮 🍢 🍡 🍧 🌰 🍠 🫔 🫓 🌰 🍠 🫔 🫓 🧆 🫕 🫒 🧃 🌶 🥜

### **مشروع يهدف الى تصنيف وصفات الطعام المدخلة الى احد السبع اصناف 🎯**
- **اطباق رئيسية 🍝**
- **مقبلات 🍟**
- **حساء 🍲**
- **سلطات 🥗**
- **ساندويتشات 🥪**
- **حلويات 🍰**
- **مشروبات 🥤**

In [50]:
import numpy as np
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.isri import ISRIStemmer
import os
from collections import Counter
from nltk import FreqDist 
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
# حذف التشويش 
def noise_removal(text):
    text = re.sub(r'<.*?>', '', text)  # حذف وسوم HTML
    text = re.sub(r'http\S+', '', text)  # حذف URLs
    text = re.sub(r'[^\w\s]', '', text)  # حذف علامات التنصيص
    text = re.sub(r'\d+', '', text)  # حذف الارقام
    return text

In [5]:
#حذف الكلمات التي لا تؤثر في المعنى
def removing_stop_words(words):

    arabic_stopwords = set(stopwords.words('arabic'))
    
    filtered_words = [word for word in words if word not in arabic_stopwords]
    return filtered_words

In [7]:
# ارجاع الكلمات لجذرها
def stemming(words):
    stemmer = ISRIStemmer()
    stemmed_words = [stemmer.stem(word) for word in words]
    return stemmed_words

In [9]:
def text_processing(text):
    text = noise_removal(text)
    tokens = word_tokenize(text)
    tokens = removing_stop_words(tokens)
    tokens = stemming(tokens)
    return tokens

In [34]:
def calculateBOW(wordset,doc):
    # انشاء قاموس للكلمات الفريده وتكراره
  fdist = FreqDist(doc)
  return {word: fdist[word] for word in wordset}

In [36]:
corpus_root = 'C:\\Users\\LENOVO\\Desktop\\recipe_classifier\\recipes_corpus'
corpus = {}

for file_name in os.listdir(corpus_root):
    #عنوان الملف
    label = file_name.split('.')[0] #حلى.txt ['حلى','txt']
    with open(os.path.join(corpus_root, file_name), 'r', encoding='utf-8') as file:
        corpus[label] = file.read()


processed_corpus = {}
for label, text in corpus.items():
    processed_corpus[label] = text_processing(text)


In [38]:
# حساب عدد تكرار الكلمات
freq = {}
for label in processed_corpus:
    freq[label] = Counter(processed_corpus[label]) 

# حساب اعلى 100 كلمة ظهرت في كل تصنيف
top_words = {}
for label in processed_corpus:
    top_words[label] = [word for word, _ in freq[label].most_common(100)]

In [40]:
# قائمة تجمع قوائم الوصفات
documents = [processed_corpus[label] for label in processed_corpus]

# قائمة بالاصناف
labels = [label for label in processed_corpus]

# قائمة للكلمات الفريدة
wordset = np.unique([word for doc in documents for word in doc])

bow_dicts = [calculateBOW(wordset, doc) for doc in documents]

In [42]:
# دالة تقارن بين التصنيفات وقاموس المستخدم

def classify_recipes(bow_dict): #يُعرّف دالة اسمها classify_recipes تأخذ قاموس bow_dict (Bag of Words) الذي يحتوي على الكلمات من وصفة وعدد تكرار كل كلمة
    scores = {} #ينشئ قاموس scores لحفظ "الدرجة" لكل تصنيف. كل تصنيف يحصل على مجموع عدد مرات ظهور كلماته المميزة.
    
    for label in top_words: #نبدأ حلقة تمر على كل تصنيف (مثلاً: "حلى"، "طبق رئيسي"، "مقبلات").
#top_words يجب أن يكون قاموس يحتوي على الكلمات المميزة لكل تصنيف.

        scores[label] = sum(bow_dict[word] for word in top_words[label] if word in bow_dict) #نحسب مجموع التكرارات للكلمات المميزة لهذا التصنيف الموجودة فعلًا في bow_dict.
#يعني: إذا كانت الوصفة تحتوي على كلمات كثيرة تنتمي إلى تصنيف "طبق رئيسي"، يحصل "طبق رئيسي" على مجموع أكبر
   
    return max(scores, key=scores.get) #ترجع التصنيف الذي حصل على أعلى درجة، أي أعلى تطابق مع كلمات الوصفة.




In [44]:
# نص المستخدم
user_text = input("أدخل خطوات وصفة لتحديد نوعها (حلى، طبق رئيسي، مقبلات، ...): ")

# معالجة نص المستخدم
user_tokens = text_processing(user_text)


# تحديد التصنيف الأنسب
predicted_class = classify_recipes(calculateBOW(wordset,user_tokens))
print("🔎 تصنيف الوصفة:", predicted_class)

KeyboardInterrupt: Interrupted by user